In [ ]:
# @title Importações e Parâmetros Globais { display-mode: "form" }
# importando bibliotecas

"""
  @description 'Python que obtem detalhes dos cliques no Short.io e insere no BigQuery'
  @author 'Everton Moraes'
  @version '1.0'
  @since '08/10/2024'

"""
import requests
import datetime
import pandas as pd
import pandas_gbq as pd_gbq
from pytz import timezone
from google.auth import default
from google.cloud import bigquery
from google.colab import auth


# Parâmetros Globais
fuso_horario = timezone('America/Sao_Paulo')

# Ambiente BigQuery
var_project = <projeto_bigquery>

# Autenticando usuario
auth.authenticate_user()

# Credenciais Pandas BQ
credentials, _ = default()
pd_gbq.context.credentials = credentials
pd_gbq.context.project = var_project

api_key= <chave api short.io>

In [ ]:
# @title Funções { display-mode: "form" }

# Variaveis Globais para as Funções

ls_log = []

# log
def log (mensagem):
  """
  Função para gera o log/acompanhamento das execuções

  Args:
    mensagem (str): O caminho da pasta atual.
  """
  agora = datetime.datetime.now().astimezone(fuso_horario).strftime("%d/%m/%Y %H:%M:%S")
  print(f'{agora} - {mensagem}')
  ls_log.append(f'{agora} - {mensagem}')

def obter_clicks_data_dominio(id_dominio,data,limite_lote):
  """
  Função obter os clicks por data do Short.io
  Documentação do método utilizado: https://developers.short.io/reference/postdomaindomainidlast_clicks

  Args:
    id_dominio(str): Id do dominio a ser obtido os clicks
    data(strg): Data a obter os clicks, formato YYYY-MM-DD

  Returns:
    dict: Retorno da API
  """

  data_inicio = datetime.datetime.strptime(data, "%Y-%m-%d")
  data_fim = data_inicio + datetime.timedelta(days=1)

  # Header
  headers = {
      "accept": "*/*",
      "content-type": "application/json",
      "Authorization": api_key
  }

  # Configura parametros
  payload = {
      "limit": limite_lote,
      "period": "custom",
      "tz": "America/Sao_Paulo",
      "startDate": data_inicio.strftime("%Y-%m-%d"),
      "endDate": data_fim.strftime("%Y-%m-%d"),
  }

  url = f"https://statistics.short.io/statistics/domain/{id_dominio}/last_clicks"

  log(f"Obtendo os cliques do dia {data_inicio.strftime('%Y-%m-%d')}, do id_dominio: {id_dominio}")

  # Realiza requisicao
  request = requests.post(url, json=payload, headers=headers)

  request.raise_for_status()
  js_retorno = request.json()
  df_retorno = pd.DataFrame(js_retorno)

  return js_retorno,df_retorno


def obter_clicks_data_dominio_browser(id_dominio,browser,data,limite_lote):
  """
  Função obter os clicks por data do Short.io
  Documentação do método utilizado: https://developers.short.io/reference/postdomaindomainidlast_clicks

  Args:
    id_dominio(str): Id do dominio a ser obtido os clicks
    browser(str): Browser a ser obtido os clicks
    data(strg): Data a obter os clicks, formato YYYY-MM-DD

  Returns:
    dict: Retorno da API
  """

  data_inicio = datetime.datetime.strptime(data, "%Y-%m-%d")
  data_fim = data_inicio + datetime.timedelta(days=1)

  # Header
  headers = {
      "accept": "*/*",
      "content-type": "application/json",
      "Authorization": api_key
  }

  # Configura parametros
  payload = {
      "limit": limite_lote,
      "period": "custom",
      "tz": "America/Sao_Paulo",
      "startDate": data_inicio.strftime("%Y-%m-%d"),
      "endDate": data_fim.strftime("%Y-%m-%d"),
      "include": {
        "browsers": [browser]
                },
  }

  url = f"https://statistics.short.io/statistics/domain/{id_dominio}/last_clicks"

  log(f"Obtendo os cliques do dia {data_inicio.strftime('%Y-%m-%d')}, do id_dominio: {id_dominio}, do browser: {browser}")

  # Realiza requisicao
  request = requests.post(url, json=payload, headers=headers)

  request.raise_for_status()
  js_retorno = request.json()
  df_retorno = pd.DataFrame(js_retorno)


  return js_retorno,df_retorno

def obter_ultima_data_bigquery():
  """
  Função para obter a ultima data dos cliques no BigQuery

  Returns:
    str: Retorno da data
  """

  # Criando um Cliente BigQuery
  bq_client = bigquery.Client(project=var_project)

  # Query verificar se existe o cod_segmetacao
  consulta = bq_client.query("SELECT MAX(CAST(A.DATAHORA_ACESSO AS DATE)) FROM `<projeto_bigquery>.<dataset_bigquery>.TB_SHORTIO_CLIQUES`  AS A")

  for row in consulta.result():
    ult_data = row[0]

  return ult_data

def obter_browsers_bigquery():
  """
  Função para obter browsers no BigQuery

  Returns:
    list: Lista de browsers
  """
  # Criando um Cliente BigQuery
  bq_client = bigquery.Client(project=var_project)

  # Query verificar se existe o browser
  consulta = bq_client.query("SELECT DISTINCT BROWSER FROM `<projeto_bigquery>.<dataset_bigquery>.TB_SHORTIO_CLIQUES` ORDER BY 1 ASC")

  ls_final = []

  for linha in consulta.result():
      ls_final.append(linha[0])

  return ls_final


def inserir_dados_bigquery(df):
  """
  Função para inserir os dados no BigQuery

  """
  # Copia Dataframe
  df_importar = df.copy()

  # df_importar a coluna com a data tratada
  df_importar['DATAHORA_ACESSO'] = pd.to_datetime(df_importar['dt'], format='%Y-%m-%dT%H:%M:%S.%f-03:00')

  # Renome a coluna proto
  df_importar = df_importar.rename(columns={'proto': 'protocolo'})
  # Reorganiza as colunas
  df_importar2 = df_importar[['DATAHORA_ACESSO'] + [col for col in df_importar.columns if col != 'DATAHORA_ACESSO']]
  df_importar = df_importar2

  log(f"Aplicado as tratativas nos dados")

  # Insere os dados no BigQuery
  pd_gbq.to_gbq(df_importar2, '<dataset_bigquery>.TB_SHORTIO_CLIQUES', project_id='<projeto_bigquery>', if_exists='append')

  log(f"Inseridos {len(df_importar):,} registros no BigQuery")

In [ ]:
# @title Execução{ form-width: "10px", display-mode: "form" }


# Configura intervalos
#inicio_intervalo  = datetime.datetime.strptime('2023-10-01', "%Y-%m-%d")
#inicio_intervalo = datetime.datetime.now(fuso_horario)- datetime.timedelta(days=1)

ult_data_bigquery = obter_ultima_data_bigquery()

# se o objeto esta vazio seta a data de um ano atrás (tempo limite de dados guardados pela short.io)
if ult_data_bigquery == None:
  data_inicio = datetime.datetime.now(fuso_horario) - datetime.timedelta(days=370)
else:
  data_inicio =  datetime.datetime.strptime(str(ult_data_bigquery), "%Y-%m-%d")   + datetime.timedelta(days=1)

inicio_intervalo  = data_inicio
fim_intervalo = datetime.datetime.now(fuso_horario) - datetime.timedelta(days=1)

"""
inicio_intervalo  = datetime.datetime.strptime('2024-05-17', "%Y-%m-%d")
fim_intervalo  = datetime.datetime.strptime('2024-05-17', "%Y-%m-%d")
"""

# Configura dominios
id_dominio = [
               ['<id_dominio_1>','<nome_dominio_1>']
              ,['<id_dominio_2>','<nome_dominio_2>']
              ,['<id_dominio_3>','<nome_dominio_3>']
              ]

# Data atual
data_atual = inicio_intervalo

# Loop datas
while data_atual.date() <= fim_intervalo.date():

  # Loop dominios
  for id in id_dominio:

    id_atual = id[0]
    dominio_atual = id[1]

    log(f"Obtendo os cliques do dominio '{dominio_atual}'")

    # Limite de cliques por lote
    limite_lote = 5000

    # Obtendo os clicks da data/dominio
    js_clicks,df_click = obter_clicks_data_dominio(id_atual,data_atual.strftime("%Y-%m-%d"),limite_lote)

    # Verifica se tiveram cliques
    if len(df_click) == 0:
      log(f"Não foi localizados cliques no dia {data_atual.strftime('%Y-%m-%d')}")

    # Localizado cliques
    if len(df_click) > 0:

      log(f"Obtido {len(df_click):,} cliques do dia {data_atual.strftime('%Y-%m-%d')}")

      # Verifica se atingiu o total do limite do lote
      if len(df_click) == limite_lote:

          # Otem os browsers exitestes no BigQuery
          ls_browsers = obter_browsers_bigquery()

          # Loop nos browsers
          for browser in ls_browsers:

            # Obtem os clicks da data/dominio/browser
            js_clicks_browser,df_click_browser =  obter_clicks_data_dominio_browser(id_atual,browser,data_atual.strftime("%Y-%m-%d"),limite_lote)

            # Verifica se tiveram cliques
            if len(df_click_browser) == 0:
              log(f"Não foi localizados cliques no dia {data_atual.strftime('%Y-%m-%d')}, do dominio {dominio_atual}, do browser {browser}")

            # Localizado cliques
            if len(df_click_browser) > 0:

              log(f"Obtido {len(df_click_browser):,} cliques do dia {data_atual.strftime('%Y-%m-%d')}, do dominio {dominio_atual}, do browser {browser}")

              # Insere registros no BigQuery
              inserir_dados_bigquery(df_click_browser)

      # Se não atingiu o total do limite do lote
      if len(df_click) < limite_lote:

          # Insere registros no BigQuery
          inserir_dados_bigquery(df_click)

  # Incrementa a data atual
  data_atual = data_atual + datetime.timedelta(days=1)